In [ ]:
import pandas as pd
import numpy as np

# 1. Define File Paths
# Adjust these to match your exact local folder structure
initial_path = '../../data/initial_data/' 
test_path = '../../data/test_data/'

def load_and_combine(filename):
    """
    Loads historical data and appends test data if available.
    Ensures Year 11 is part of the dataframe for feature generation.
    """
    # Load historical data
    df = pd.read_csv(f"{initial_path}{filename}")
    
    # Try to load test data
    try:
        df_test = pd.read_csv(f"{test_path}{filename}")
        # Concatenate
        df = pd.concat([df, df_test], ignore_index=True)
    except FileNotFoundError:
        print(f"Note: No test file found for {filename}")
        pass
    return df

# Load all relevant tables
coaches = load_and_combine('coaches.csv')
teams = load_and_combine('teams.csv')
players_teams = load_and_combine('players_teams.csv')
awards = pd.read_csv(f"{initial_path}awards_players.csv")

# Sort for consistent lag calculation
teams = teams.sort_values(['tmID', 'year'])
coaches = coaches.sort_values(['tmID', 'year'])

print(f"Teams loaded: {teams.shape}")
print(f"Coaches loaded: {coaches.shape}")

In [ ]:
# --- Feature Engineering: Team Expectations ---

# 1. Calculate derived metrics for TRAINING years (where we have stats)
# Pythagorean Expectation Formula for Basketball (exponent ~13.91)
# We use a small epsilon to avoid division by zero
teams['o_pts'] = teams['o_pts'].fillna(0)
teams['d_pts'] = teams['d_pts'].fillna(0)

# Real Win %
teams['win_pct'] = teams['won'] / (teams['won'] + teams['lost'])

# Expected Win % based on points scored/allowed
teams['expected_win_pct'] = (teams['o_pts']**13.91) / \
                            ((teams['o_pts']**13.91) + (teams['d_pts']**13.91) + 1e-9)

# Luck Factor: Did they win more than they should have?
# High luck often implies regression next year
# Low luck implies bounce-back.
teams['luck_factor'] = teams['win_pct'] - teams['expected_win_pct']

# 2. SHIFT metrics to create "Previous Season" features
# For Year 11, this pulls Year 10 stats into the row.
teams['prev_win_pct'] = teams.groupby('tmID')['win_pct'].shift(1)
teams['prev_expected_win_pct'] = teams.groupby('tmID')['expected_win_pct'].shift(1)
teams['prev_luck_factor'] = teams.groupby('tmID')['luck_factor'].shift(1)

# Fill NaNs for the very first season of a team (usually average or 0)
teams['prev_win_pct'] = teams['prev_win_pct'].fillna(0.5)
teams['prev_expected_win_pct'] = teams['prev_expected_win_pct'].fillna(0.5)
teams['prev_luck_factor'] = teams['prev_luck_factor'].fillna(0)

# Select only the features we need for the merge later
team_features = teams[['year', 'tmID', 'prev_win_pct', 'prev_expected_win_pct', 'prev_luck_factor']].copy()

print("Team features calculated. Head:")
display(team_features.tail())

In [ ]:
# --- Feature Engineering: Roster Talent ---

# Calculate Player Efficiency in previous years
pt_stats = players_teams.copy()
cols_to_fix = ['points', 'rebounds', 'assists', 'steals', 'blocks', 
               'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'turnovers']
pt_stats[cols_to_fix] = pt_stats[cols_to_fix].fillna(0)

pt_stats['efficiency'] = (
    pt_stats['points'] + pt_stats['rebounds'] + pt_stats['assists'] + 
    pt_stats['steals'] + pt_stats['blocks'] - 
    (pt_stats['fgAttempted'] - pt_stats['fgMade']) - 
    (pt_stats['ftAttempted'] - pt_stats['ftMade']) - 
    pt_stats['turnovers']
)

# Create a lookup for Player Efficiency in Year Y
eff_lookup = pt_stats[['playerID', 'year', 'efficiency']].copy()
eff_lookup['next_year'] = eff_lookup['year'] + 1  # This stat applies to the NEXT season
eff_lookup = eff_lookup[['playerID', 'next_year', 'efficiency']]

# Map efficiency to the Current Roster (Year T)
# We join the Roster of Year T with the Efficiency of Year T-1
roster = players_teams[['playerID', 'year', 'tmID']]
roster_with_talent = roster.merge(
    eff_lookup, 
    left_on=['playerID', 'year'], 
    right_on=['playerID', 'next_year'], 
    how='left'
)

# Sum up efficiency per team to get "Aggregate Talent"
talent_score = roster_with_talent.groupby(['tmID', 'year'])['efficiency'].sum().reset_index()
talent_score.rename(columns={'efficiency': 'talent_agg'}, inplace=True)

# Normalize and Create "Turnaround Potential"
feature_df = team_features.merge(talent_score, on=['tmID', 'year'], how='left')

# Fill missing talent (e.g. Year 1 or new teams) with mean
feature_df['talent_agg'] = feature_df['talent_agg'].fillna(feature_df['talent_agg'].mean())

# Z-Score Normalization PER YEAR
feature_df['talent_zscore'] = feature_df.groupby('year')['talent_agg'].transform(lambda x: (x - x.mean()) / x.std())
feature_df['prev_wins_zscore'] = feature_df.groupby('year')['prev_win_pct'].transform(lambda x: (x - x.mean()) / x.std())

feature_df['talent_zscore'] = feature_df['talent_zscore'].fillna(0)
feature_df['prev_wins_zscore'] = feature_df['prev_wins_zscore'].fillna(0)

# The Narrative Metric: High Talent (Z > 0) + Low Past Wins (Z < 0) = High Potential
feature_df['turnaround_potential'] = feature_df['talent_zscore'] - feature_df['prev_wins_zscore']

display(feature_df.head())
print("Talent features ready.")

In [ ]:
# --- Feature Engineering: Coach Narrative ---

# 1. Identify Main Coach (stint 0 or 1, usually 0 is head coach at start)
main_coaches = coaches[(coaches['stint'] == 0)].copy()

# 2. Calculate Tenure (How long have they been at THIS team?)
# We group by Team and create a cumulative count of years
main_coaches['coach_tenure'] = main_coaches.groupby(['tmID', 'coachID']).cumcount() + 1

# 3. Calculate Past Awards (Voter Fatigue)
# Did this coach win COTY before?
coty_history = awards[awards['award'] == 'Coach of the Year'].copy()
coty_history['is_coty'] = 1
coty_history = coty_history[['playerID', 'year', 'is_coty']].rename(columns={'playerID': 'coachID'})

# Cumulative sum of past awards (Shifted, so we only know about *past* awards)
# We create a full skeleton of coach/year to avoid missing years breaking the cumsum
coach_years = main_coaches[['coachID', 'year']].drop_duplicates()
coach_years = coach_years.merge(coty_history, on=['coachID', 'year'], how='left').fillna(0)

# Calculate cumulative awards, then shift by 1 to represent "awards won PRIOR to this season"
coach_years['prev_awards'] = coach_years.groupby('coachID')['is_coty'].cumsum().shift(1).fillna(0)

# 4. Merge Coach Features into Main DataFrame
final_df = feature_df.merge(main_coaches[['year', 'tmID', 'coachID', 'coach_tenure']], on=['year', 'tmID'], how='inner')
final_df = final_df.merge(coach_years[['year', 'coachID', 'prev_awards']], on=['year', 'coachID'], how='left')

# 5. Add Target Variable (Current Year COTY)
final_df = final_df.merge(coty_history, on=['year', 'coachID'], how='left')
final_df['is_coty'] = final_df['is_coty'].fillna(0).astype(int)

# Final Cleanup
final_df['prev_awards'] = final_df['prev_awards'].fillna(0)

print(f"Final Dataset Shape: {final_df.shape}")
display(final_df.head())

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss

# --- 1. Setup Models & Metrics ---

features = [
    'prev_win_pct', 
    'prev_expected_win_pct', 
    'prev_luck_factor', 
    'talent_zscore', 
    'turnaround_potential', 
    'coach_tenure', 
    'prev_awards'
]
target = 'is_coty'

# Initialize a list to store results for every year and model
all_results = []

# Define Stacking Estimators
estimators = [
    ('xgb', XGBClassifier(n_estimators=500, max_depth=4, learning_rate=0.05, 
                          subsample=0.8, eval_metric='auc', random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=42)),
    ('svc', SVC(probability=True, kernel='rbf', class_weight='balanced', random_state=42))
]

# Define the models to compare
models = {
    'XGBoost': XGBClassifier(
        n_estimators=500, max_depth=4, learning_rate=0.05, 
        subsample=0.8, colsample_bytree=0.8, eval_metric='auc', random_state=42
    ),
    'LogisticReg': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=42),
    'GaussianProcess': GaussianProcessClassifier(kernel=1.0 * RBF(1.0, length_scale_bounds=(1e-7, 1e7)), random_state=42),
    'StackingEnsemble': StackingClassifier(
        estimators=estimators,
        final_estimator=LogisticRegression(class_weight='balanced'),
        cv=3
    )
}

# --- 2. Walk-Forward Validation Loop ---
# We start from year 4 to ensure we have enough training history
for test_year in range(4, 11):
    print(f"\nProcessing Year {test_year}...")
    
    # Split Data
    train_df = final_df[final_df['year'] < test_year].copy()
    test_df = final_df[final_df['year'] == test_year].copy()
    
    # Skip years with no data or no winner defined
    if test_df.empty or test_df['is_coty'].sum() == 0:
        continue

    X_train = train_df[features]
    y_train = train_df[target]
    X_test = test_df[features]
    y_test = test_df[target]
    
    # Scale data (Important for Logistic Regression)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train and Evaluate Each Model
    for name, model in models.items():
        # Tree models handle unscaled data better/natively
        if name in ['XGBoost', 'RandomForest']:
            # Dynamic weighting for XGBoost
            ratio = (len(y_train) - sum(y_train)) / sum(y_train)
            if hasattr(model, 'set_params'):
                try:
                    model.set_params(scale_pos_weight=ratio)
                except:
                    pass # RF doesn't use scale_pos_weight
            model.fit(X_train, y_train)
            probs = model.predict_proba(X_test)[:, 1]
        
        # Distance/Probability models need scaling
        else:
            model.fit(X_train_scaled, y_train)
            probs = model.predict_proba(X_test_scaled)[:, 1]
            
        # Store predictions in a temp df for ranking
        temp_res = test_df.copy()
        temp_res['prob'] = probs
        
        # --- Calculate Metrics for this Year ---
        
        # Sort candidates by probability
        ranked = temp_res.sort_values('prob', ascending=False).reset_index(drop=True)
        
        # Find rank of actual winner (Index + 1)
        winner_rank = ranked[ranked['is_coty'] == 1].index[0] + 1
        
        # Metrics
        is_top1 = 1 if winner_rank == 1 else 0
        is_top3 = 1 if winner_rank <= 3 else 0
        mrr = 1 / winner_rank
        # Log Loss (for single year)
        ll = log_loss(y_test, probs, labels=[0,1])
        
        all_results.append({
            'Year': test_year,
            'Model': name,
            'Winner_Rank': winner_rank,
            'Top1_Acc': is_top1,
            'Top3_Acc': is_top3,
            'MRR': mrr,
            'LogLoss': ll
        })



In [ ]:
# --- 3. Final Summary & Comparison ---

results_df = pd.DataFrame(all_results)

# Group by Model to get averages
summary = results_df.groupby('Model').agg({
    'Winner_Rank': 'mean',
    'Top1_Acc': 'mean',
    'Top3_Acc': 'mean',
    'MRR': 'mean',
    'LogLoss': 'mean'
}).reset_index()

# Rename for presentation
summary = summary.rename(columns={
    'Winner_Rank': 'Avg Winner Rank',
    'Top1_Acc': 'Top-1 Accuracy',
    'Top3_Acc': 'Top-3 Accuracy',
    'MRR': 'Mean Reciprocal Rank',
    'LogLoss': 'Log Loss (Lower is Better)'
})

print("\n🏆 Model Comparison Summary 🏆")
display(summary.sort_values('Mean Reciprocal Rank', ascending=False).reset_index(drop=True).style
    .background_gradient(cmap='YlGn_r', subset=['Avg Winner Rank'], vmin=1, vmax=summary['Avg Winner Rank'].max() if summary['Avg Winner Rank'].max() > 1 else 2) # Assuming 1 is the best rank, higher is worse. Max can be variable.
    .background_gradient(cmap='YlGn', subset=['Top-1 Accuracy'], vmin=0, vmax=1)
    .background_gradient(cmap='YlGn', subset=['Top-3 Accuracy'], vmin=0, vmax=1)
    .background_gradient(cmap='YlGn', subset=['Mean Reciprocal Rank'], vmin=0, vmax=1)
    .background_gradient(cmap='YlGn_r', subset=['Log Loss (Lower is Better)'], vmin=0, vmax=summary['Log Loss (Lower is Better)'].max())
)

for model in models:
    print(f"\n--- Year-by-Year Performance ({model}) ---")
    display(results_df[results_df['Model'] == model][['Year', 'Winner_Rank', 'Top1_Acc', 'Top3_Acc', 'MRR']])

In [ ]:
# 4. --- Final Prediction for Year 11 ---

# Train on ALL history (Years 1-10)
train_df_final = final_df[final_df['year'] < 11].copy()
test_df_final = final_df[final_df['year'] == 11].copy()

X_train_final = train_df_final[features]
y_train_final = train_df_final[target]
X_test_final = test_df_final[features]

# Create Scaled Versions for Non-Tree Models (LogReg, SVC, GaussianProcess)
scaler_final = StandardScaler()
X_train_scaled = scaler_final.fit_transform(X_train_final)
X_test_scaled = scaler_final.transform(X_test_final)

print(f"{'='*30} YEAR 11 PREDICTIONS {'='*30}")

# Iterate, train and predict
for name, model in models.items():
    print(f"\n⚙️  Model: {name}")
    
    # Select Data Type based on Model
    if name in ['XGBoost', 'RandomForest']:
        # Update scale_pos_weight for XGBoost to reflect full dataset imbalance
        ratio = (len(y_train_final) - sum(y_train_final)) / sum(y_train_final)
        if hasattr(model, 'set_params'):
            try:
                model.set_params(scale_pos_weight=ratio)
            except:
                pass 
        
        model.fit(X_train_final, y_train_final)
        probs = model.predict_proba(X_test_final)[:, 1]
        
    else:
        # Distance/Linear models use scaled data
        model.fit(X_train_scaled, y_train_final)
        probs = model.predict_proba(X_test_scaled)[:, 1]
    
    # Identify the Winner
    # Assign probabilities to a temp dataframe to find the max
    test_df_final['coty_prob'] = probs
    top_pick = test_df_final.sort_values('coty_prob', ascending=False).iloc[0]
    
    # Print Formatted Result
    print(f"   🏆 Prediction:  {top_pick['coachID']} ({top_pick['tmID']})")
    print(f"   📈 Probability: {top_pick['coty_prob']:.2%}")
    print(f"   📖 Narrative:   Turnaround Potential = {top_pick['turnaround_potential']:.2f}")